# Introduction
## Question
How have the lyrics of the most popular songs in the US changed over time?
## Why Does It Matter?
Music tends to reflect the current state of society. As our society has gotten more progressive and open, so have our popular music lyrics. By looking at how lyrics have changed since 1950, an arugment can be made regarding what is considered acceptable in the United States. For example, if technology is mentioned more often, it could be reflecting the rise of technology in our society. Additionally, an increase in lyrics about provactive actions can indicate that our society has relaxed our standards around sex. By analyzing different themes in in popular music lyrcis, commentary on the values of our society can argued.
# Sources
I am using a csv file created by Eunice Chan (https://github.com/eunice-chan/DH_PROJ_Billboard) that details the Billboard Top 100 from 1950-2018. Additionally, much of the intial data cleaning, formatting, and basic analysis is taken from Chan's project. Chan manually inputted lyrics, but I am hoping to webscrape so that I can have a larger library of words.  

I will be using Rishabh Parekh's project(https://github.com/rishabh-parekh/dh100final) as a guideline on how to webscrape from genius.com. While Parekh only found the lyrics for four songs using this technique, I am hoping to adapt it to be able to webscrape lyrics for all the Billboard Top 100 Songs 1950-2018. 

Furthermore, I have found inspiration for this project from a Medium article (https://medium.com/the-omnivore/the-evolution-of-the-american-pop-lyric-61ef31b24f03) that studied a similar question. I will be expanding on this by including more association words and looking at more themes. 

# Workflow
1. Upload 'Billboard100(1950-2018).csv'created by Eunice Chan to Jupyter Notebook.
2. Drop the manually added lyrics column and webscrape for all lyrics using technique by Rishabh Parekh.
3. Perform basic data analysis with influence from Chan's work in Jupyter Notebook.
## CTA Analysis:

1. Preprocessing Lyrics without punctuation, lyrics by phrases, lyrics by words.
2. Basic: Unique artists in decade, unique artists, number of times artists appear, unique phrases, average character count, average word count.
3. Frequency: Parts of speech distribution (by decade), sepculative words, personal pronouns (total, first vs second, first person singular vs first person plural, genders), lexical diversity.
4. Research Question Analysis: Analysis the frequency of words associated with chosen themes. 
5. Network graph
6. Sentiment analysis (by phrases)
7. Common words: direct count (without stopwords), TF-IDF, topic modeling

# Assumptions & Concerns
I am currently strugging to webscrape lyrics for all of the songs. It seems to be mainly a debugging issue that I should be able to get resolved. Once I fix that, I will need to ensure that the webscraping worked for all of the songs. In the case that not all lyrics could not be webscraped, I will need to ensure that every year (or decade) has the same number of songs to ensure that there is not unexpected bias. Furthermore, it will be difficult to idenitfy every association word for a theme. It is possible that a word could be referencing another theme or that it could be completeley unrelated in certain cases. 
    
# Tools & Methods
## Preliminary Importing

In [ ]:
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from wordcloud import WordCloud

from collections import Counter
from string import punctuation

import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import random 
import os
import re

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use("seaborn-pastel")

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Data

In [ ]:
link = 'https://drive.google.com/file/d/1bEqtahe39bMIIafkoI07Hp70KUXzcmr6/view?usp=sharing'
import pandas as pd
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile("alisha data.csv")  
  
full_music = pd.read_csv("alisha data.csv")
print(full_music)

     Position  ...                                             Lyrics
0           1  ...  Irene, goodnight\nIrene, goodnight\nGoodnight,...
1           2  ...  Mona lisa, mona lisa, men have named you\nyou'...
2           3  ...                                                NaN
3           4  ...                                                NaN
4           5  ...                                                NaN
...       ...  ...                                                ...
6311       96  ...                                                NaN
6312       97  ...                                                NaN
6313       98  ...                                                NaN
6314       99  ...                                                NaN
6315      100  ...                                                NaN

[6316 rows x 5 columns]


In [ ]:
# #This is the saved csv file from Google Sheets with lyrics added to web-scrapped data
# full_music = pd.read_csv("alisha data.csv")
# pd.to_numeric(full_music["Year"])
# top30 = full_music.loc[full_music["Position"].isin([str(i) for i in np.arange(1, 30)])].reset_index(drop=True)
# #Get the songs from the Billboard entries every 5 years
# #subset_music = full_music.loc[full_music["Year"]%5 == 0]
# #subset_music = full_music
# #Only get the #1 and #2 song in each Billboard entry
# #data = subset_music.loc[subset_music["Position"].isin(["1", "2"])].reset_index(drop=True)
# #print(data.shape)
# full_music

In [ ]:
import requests
from bs4 import BeautifulSoup
defaults = {
    'request': {
        'token': 'i507Djjp1UurKonxzH_iyuCAP1Af_zSAjQ_eM7fZDV_ucIoEIhACnGMU3tT6IhDz',
        'base_url': 'https://api.genius.com'
    },
    'message': {
        'search_fail': 'The lyrics for this song were not found!',
        'wrong_input': 'Wrong number of arguments.\n' \
                       'Use two parameters to perform a custom search ' \
                       'or none to get the song currently playing on Spotify.'
    }
}

In [ ]:
def request_song_info(song_title, artist_name):
    base_url = defaults['request']['base_url']
    headers = {'Authorization': 'Bearer ' + defaults['request']['token']}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

In [ ]:
# def scrap_song_url(url):
#     print(url)
#     page = requests.get(url)
#     html = BeautifulSoup(page.text, 'html.parser')
#     [h.extract() for h in html('script')]
#     lyrics = html.find('div', class_='lyrics').get_text()
#     return lyrics

In [ ]:
def scrap_song_url(url):
#     print(url)
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    [h.extract() for h in html('script')]
    lyrics = html.select('div[class*="Lyrics__Container"]')
    
    lyrics_out = []
    for lyric in lyrics:
        for br in lyric.find_all("br"):
            br.replace_with("\n")
        lyric_text = lyric.get_text(strip = False)
        lyrics_out.append(lyric_text)
    return '\n'.join(lyrics_out)

scrap_song_url('https://genius.com/Anton-karas-the-third-man-theme-lyrics')

''

In [ ]:
lyrics = scrap_song_url('https://genius.com/Nat-king-cole-l-o-v-e-lyrics')

In [ ]:
print(lyrics)

[Verse 1]
L is for the way you look at me
O is for the only one I see
V is very, very extraordinary
E is even more than anyone that you adore can

[Chorus]
Love is all that I can give to you
Love is more than just a game for two
Two in love can make it, take my heart and please don't break it
Love was made for me and you

[Verse 2]
L is for the way you look at me
O is for the only one I see
V is very, very extraordinary
E is even more than anyone that you adore can

[Hook]
Love is all that I can give to you
Love is more than just a game for two
Two in love can make it, take my heart and please don't break it
Love was made for me and you

[Outro]
Love a-was made for me and you
Love a-was made for me and you


In [ ]:
def get_song_lyrics(song_title,artist_name):
    lyrics_dict= {}
    lyrics_df = pd.DataFrame()
    response = request_song_info(song_title, artist_name)
    json = response.json()
    remote_song_info = None
    for hit in json['response']['hits']:
        if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
            remote_song_info = hit
            break
    # Extract lyrics from URL if song was found
    if remote_song_info is not None:
        song_url = remote_song_info['result']['url']
        lyrics = scrap_song_url(song_url)
        lyrics_dict.update({"artist_name" : artist_name, "song_title" : song_title, "lyrics": lyrics})
        lyrics_df = lyrics_df.append(lyrics_dict, ignore_index=True)
        ly_file = song_title+"_"+artist_name+".csv"; 
        lyrics_df.to_csv(ly_file,index=False)
    else:
        print(defaults['message']['search_fail'])
    return lyrics_df

In [ ]:
no_lyrics = full_music.drop(columns = ['Lyrics'])
no_lyrics

,Position,Song,Artist,Year
0,1,"""Goodnight Irene""",Gordon Jenkins & The Weavers,1950
1,2,"""Mona Lisa""",Nat King Cole,1950
2,3,"""Third Man Theme""",Anton Karas,1950
3,4,"""Sam's Song""",Gary & Bing Crosby,1950
4,5,"""Simple Melody""",Gary & Bing Crosby,1950
...,...,...,...,...
6311,96,"""One Number Away""",Luke Combs,2018
6312,97,"""Powerglide""",Rae Sremmurd featuring Juicy J,2018
6313,98,"""IDGAF""",Dua Lipa,2018
6314,99,"""Mi Gente""",J Balvin and Willy William featuring Beyoncé,2018


In [ ]:
song_artist = no_lyrics.drop(columns = ['Position','Year'])
song_artist

,Song,Artist
0,"""Goodnight Irene""",Gordon Jenkins & The Weavers
1,"""Mona Lisa""",Nat King Cole
2,"""Third Man Theme""",Anton Karas
3,"""Sam's Song""",Gary & Bing Crosby
4,"""Simple Melody""",Gary & Bing Crosby
...,...,...
6311,"""One Number Away""",Luke Combs
6312,"""Powerglide""",Rae Sremmurd featuring Juicy J
6313,"""IDGAF""",Dua Lipa
6314,"""Mi Gente""",J Balvin and Willy William featuring Beyoncé


In [ ]:
song_artist['Song'] = song_artist['Song'].str.replace("\"", "")
song_artist

,Song,Artist
0,Goodnight Irene,Gordon Jenkins & The Weavers
1,Mona Lisa,Nat King Cole
2,Third Man Theme,Anton Karas
3,Sam's Song,Gary & Bing Crosby
4,Simple Melody,Gary & Bing Crosby
...,...,...
6311,One Number Away,Luke Combs
6312,Powerglide,Rae Sremmurd featuring Juicy J
6313,IDGAF,Dua Lipa
6314,Mi Gente,J Balvin and Willy William featuring Beyoncé


In [ ]:
song_listed = song_artist['Song'].tolist()

In [ ]:
artist_list = song_artist['Artist']
artist_list = artist_list.str.replace('featuring.*',"").to_list()

In [ ]:
combined = list(zip(song_listed, artist_list))
combined

[('Goodnight Irene', 'Gordon Jenkins & The Weavers'),
 ('Mona Lisa', 'Nat King Cole'),
 ('Third Man Theme', 'Anton Karas'),
 ("Sam's Song", 'Gary & Bing Crosby'),
 ('Simple Melody', 'Gary & Bing Crosby'),
 ('Music, Music, Music', 'Teresa Brewer'),
 ('Third Man Theme', 'Guy Lombardo'),
 ('Chattanoogie Shoe Shine Boy', 'Red Foley'),
 ('Harbor Lights', 'Sammy Kaye'),
 ("It Isn't Fair", 'Sammy Kaye & Don Cornell'),
 ("If I Knew You Were Coming I'd have Baked a Cake", 'Eileen Barton'),
 ("Bonaparte's Retreat", 'Kay Starr'),
 ('Tzena, Tzena, Tzena', 'Gordon Jenkins & The Weavers'),
 ("There's No Tomorrow", 'Tony Martin'),
 ('The Thing', 'Phil Harris'),
 ('Sentimental Me', 'Ames Brothers'),
 ('I Wanna Be Loved', 'Andrews Sisters & Gordon Jenkins'),
 ('Tennessee Waltz', 'Patti Page'),
 ("I Can Dream, Can't I", 'Andrews Sisters & Gordon Jenkins'),
 ("I'll Never Be Free", 'Tennessee Ernie Ford & Kay Starr'),
 ('All My Love', 'Patti Page'),
 ('My Foolish Heart', 'Gordon Jenkins'),
 ('Rag Mop', 'A

In [ ]:
song_list = combined #how do i correctly format this to create list of song and artist for every song in csv??
# Search for matches in request response
all_lyrics_df = pd.DataFrame()
for song_title, artist_name in song_list:
    # File with Lyrics data. 
    print(song_title)
    print(artist_name) 
    ly_file = "data/" + song_title+"_"+artist_name+".csv"; 
    if os.path.isfile(ly_file):
        lyrics_df = pd.read_csv(ly_file)
    else:
        lyrics_df = get_song_lyrics(song_title,artist_name)
    all_lyrics_df = all_lyrics_df.append(lyrics_df, ignore_index=True)
print(all_lyrics_df.shape)

Streaming output truncated to the last 5000 lines.
I'll Be
Foxy Brown 
The lyrics for this song were not found!
If It Makes You Happy
Sheryl Crow
Never Make a Promise
Dru Hill
When You Love a Woman
Journey
Up Jumps da Boogie
Timbaland & Magoo 
The lyrics for this song were not found!
I Don't Want To
Toni Braxton
Everyday Is a Winding Road
Sheryl Crow
Cold Rock a Party
MC Lyte
Pony
Ginuwine
Building a Mystery
Sarah McLachlan
I Love You Always Forever
Donna Lewis
Your Woman
White Town
C U When U Get There
Coolio 
Change the World
Eric Clapton
My Baby Daddy
B-Rock and the Bizz
The lyrics for this song were not found!
Tubthumping
Chumbawamba
Gotham City
R. Kelly
Last Night
Az Yet
ESPN Presents The Jock Jam
Various Artists
The lyrics for this song were not found!
Big Daddy
Heavy D
What About Us
Total
Smile
Scarface 
The lyrics for this song were not found!
What's on Tonight
Montell Jordan
Secret Garden
Bruce Springsteen
The One I Gave My Heart To
Aaliyah
Fly Like an Eagle
Seal
No Time
Lil' 

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
all_lyrics_df.to_csv('results.csv')
!cp results.csv "drive/My Drive/"

In [ ]:
all_lyrics_df

,artist_name,lyrics,song_title
0,Nat King Cole,,Mona Lisa
1,Anton Karas,,Third Man Theme
2,Teresa Brewer,Put another nickel in\nIn the nickelodeon\nAll...,"Music, Music, Music"
3,Guy Lombardo,When a zither starts to play\nYou'll remember ...,Third Man Theme
4,Red Foley,,Chattanoogie Shoe Shine Boy
...,...,...,...
5153,XXXTentacion,"[Chorus: XXXTENTACION]\nMmm, baby, I don't und...",Changes
5154,Brett Young,,Mercy
5155,Luke Combs,,One Number Away
5156,Dua Lipa,,IDGAF
